In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis


In [2]:
model_type = "ffn"
sys.argv = ["", "-cuda", "-model_type", model_type]
args = get_args()
outdir_suffix = "dev3"
args.n_epochs = 30 
args.init_model = "{}-qed1".format(model_type)
# args.init_model = "{}-test1".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 200

args.penalty_gumbel = False 

args.n_hidden = 400

# for long runs
args.scale_lambdas = True
args.connectivity = True 
args.valency = True 
args.euler_characteristic_penalty = True
args.conn_lambda_start = 0.001
args.conn_lambda_end = 3
args.conn_lambda_epochs_start = 12 
args.conn_lambda_epochs_end = 20
args.valency_lambda_start = 0.001
args.valency_lambda_end = 5
args.valency_lambda_epochs_start = 12
args.valency_lambda_epochs_end = 20
args.euler_lambda_start = 0.001 
args.euler_lambda_end = 2 
args.euler_lambda_epochs_start = 12 
args.euler_lambda_epochs_end = 20
args.tau_start = 1
args.tau_end = 0.01
args.tau_epochs_start = 1
args.tau_epochs_end = 16

args.cross_att_use = False
args.cross_att_random = False
args.cross_att_n_sinkhorn = 2
args.cross_att_use_gcn2 = False
args.cross_att_sigmoid = False
args.ot_lambda = 0


args.morgan_bits = 0

args.conn_penalty_function = "capped_logdet" 

print(args, '\n')

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, batch_size=200, conn_lambda_end=3, conn_lambda_epochs_end=20, conn_lambda_epochs_start=12, conn_lambda_start=0.001, conn_penalty_function='capped_logdet', connectivity=True, connectivity_hard=False, cross_att_dim=150, cross_att_n_sinkhorn=2, cross_att_random=False, cross_att_sigmoid=False, cross_att_use=False, cross_att_use_gcn2=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=True, euler_lambda_end=2, euler_lambda_epochs_end=20, euler_lambda_epochs_start=12, euler_lambda_start=0.001, ffn_activation='LeakyReLU', fgw_atoms=True, fgw_bonds=True, init_decoder_model='ffn-qed1_decode', init_model='ffn-qed1', linear_out=False, max_num_atoms=70, model_type='ffn', morgan_bits=0, n_epochs=30, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=400, n_labels=1, n_layers=5, one_batch_train=False, ot_lambda=0, ot_solver='

In [3]:
# model_type = "molemb"
# sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
# args = get_args()
# outdir_suffix = "dev3"
# args.n_epochs = 15000 
# args.init_model = "{}-morgan-test1".format(model_type)
# # args.init_model = "{}-test1".format(model_type)
# args.init_decoder_model = "{}_decode".format(args.init_model)
# args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
# args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
# args.batch_size = 50 

# args.penalty_gumbel = False 

# args.scale_lambdas = True
# args.connectivity = True 
# args.valency = True 
# args.euler_characteristic_penalty = True
# args.conn_lambda_start = 0.001
# args.conn_lambda_end = 3
# args.conn_lambda_epochs_start = 4001
# args.conn_lambda_epochs_end = 12000
# args.valency_lambda_start = 0.001
# args.valency_lambda_end = 5
# args.valency_lambda_epochs_start = 4001
# args.valency_lambda_epochs_end = 12000
# args.euler_lambda_start = 0.001 
# args.euler_lambda_end = 2 
# args.euler_lambda_epochs_start = 4001
# args.euler_lambda_epochs_end = 12000
# args.tau_start = 1
# args.tau_end = 0.01
# args.tau_epochs_start = 1
# args.tau_epochs_end = 12000

# args.morgan_bits = 1024

# args.conn_penalty_function = "capped_logdet" 

# print(args, '\n')

In [4]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs_split", args.batch_size)
# train_data_loader = get_loader("molgen/data/chembl50", "train", args.batch_size, same_number_atoms = True)
val_data_loader = get_loader("iclr19-graph2graph/data/qed", "val_pairs_split", args.batch_size)

# train_data_loader = get_loader("molgen/data/chembl50", "train_split", args.batch_size, same_number_atoms = True)
# val_data_loader = get_loader("molgen/data/chembl50", "val_split", args.batch_size, same_number_atoms = True)
# val_data_loader = None

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

No model ffn-qed1 found in mol_opt/output_dev3/ffn-qed1! Starting from scratch.
Penalty params: tau=1.00000 conn_l=0.00032 val_l=0.00030 euler_l=0.00034 epoch=-1 mode=[0 0 0] conn=True val=True euler=True
Epoch: 1


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:398: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


Losses Batch 0, train
 fgw:3.5472052
 conn_penalty:0.0000000
 val_penalty:496.6563672
 euler_penalty:144.6498145
 total:3.7431479
Penalty params: tau=1.00000 conn_l=0.00032 val_l=0.00030 euler_l=0.00034 epoch=1 mode=[0 0 0] conn=False val=False euler=False
Losses Batch 30, train
 fgw:23.0488861
 conn_penalty:0.0000000
 val_penalty:16.0211731
 euler_penalty:4.6661230
 total:23.0552068
Penalty params: tau=1.00000 conn_l=0.00032 val_l=0.00030 euler_l=0.00034 epoch=1 mode=[0 0 0] conn=False val=False euler=False
Losses Batch 60, train
 fgw:11.9575129
 conn_penalty:0.0000000
 val_penalty:8.1419077
 euler_penalty:2.3713084
 total:11.9607250
Penalty params: tau=1.00000 conn_l=0.00032 val_l=0.00030 euler_l=0.00034 epoch=1 mode=[0 0 0] conn=False val=False euler=False
Losses Batch 90, train
 fgw:8.1534231
 conn_penalty:0.0000000
 val_penalty:5.4577623
 euler_penalty:1.5895584
 total:8.1555764
Penalty params: tau=1.00000 conn_l=0.00032 val_l=0.00030 euler_l=0.00034 epoch=1 mode=[0 0 0] conn=Fals

Losses Batch 0, val
 fgw:0.3747324
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.3747324
Losses Batch 30, val
 fgw:0.3736348
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.3736348
Losses Epoch 2, val
 fgw:0.3730891
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.3730891
Epoch duration: 2759.4115471839905
Epoch: 3
Losses Batch 0, train
 fgw:0.3770721
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.3770721
Penalty params: tau=0.54117 conn_l=0.00032 val_l=0.00030 euler_l=0.00034 epoch=3 mode=[0 0 0] conn=False val=False euler=False
Losses Batch 30, train
 fgw:0.3732325
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.3732325
Penalty params: tau=0.54117 conn_l=0.00032 val_l=0.00030 euler_l=0.00034 epoch=3 mode=[0 0 0] conn=False val=False euler=False
Losses Batch 60, train
 fgw:0.3717893
 conn_penalty:0.0000000
 val_penalty:0

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

In [ ]:
for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

from mol_opt.ot_utils import FGW 
fgw_loss = FGW(alpha = 0.5)
fgw_loss(*pred_pack, tau = 1)

In [ ]:
torch.optim.Adam([torch.autograd.Variable(torch.Tensor([0.]))]).param_groups[0]['lr']

In [ ]:
0.9 ** (1/5)

In [ ]:
vars(args)

In [ ]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("/home/octav/gitrepos/tum-thesis/mol_opt/dev7")

In [ ]:
tags = [
    "pointwise10-dev8/train_avg_euler_error/mean",
    "pointwise10-dev8/train_avg_euler_error/bot_band",
    "pointwise10-dev8/train_avg_euler_error/top_band"
]
# writer.add_custom_scalars_marginchart(tags)


In [ ]:
# writer.add_scalar("plm2/plm2", 0, 1)
# writer.add_scalar("plm2/plm2", 1, 2)
# writer.add_scalar("plm2/plm2", 0, 3)
# writer.add_scalar("plm2/plm2", 1, 4)
tags = ["plm", "plm", "plm"]
# layout = {"plm" : {"plm" : ["Margin", tags]}}
layout = {"plm1" : {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]},
          "plm2": {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]}}
writer.add_custom_scalars(layout)

In [ ]:
writer._get_file_writer().add_summary()

In [ ]:
writer.logdir

In [ ]:
writer.close()

In [ ]:
import tensorboardX
tensorboardX.__version__

In [ ]:
from tensorboardX.summary import custom_scalars

In [ ]:
writer._get_file_writer().add_summary(custom_scalars(layout), global_step = 3)
writer.flush()

In [ ]:
writer.add_scalars('plm', {'plm0' : 0.0, 'plmt' : 0.1, 'plmb' : -0.1}, 3)
writer.flush()

In [ ]:
tau = 1
symbols_nll, charges_nll, bonds_nll = F.gumbel_softmax(tau = tau, dim=1, logits = symbols_logits), F.gumbel_softmax(tau=tau,dim=1,logits=charges_logits), F.gumbel_softmax(tau=tau,dim=1, logits = bonds_logits)

In [ ]:
np.sort(symbols_nll.mean(axis = 0).detach().numpy())

In [ ]:
list(molopt.opt0.parameters())[0]

In [ ]:
molopt.state_dict()

In [ ]:
model_dict = torch.load('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8')

In [ ]:
model_dict["model"]['opt0.weight']

In [ ]:
molopt2 = MolOpt(args)

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt2.load_state_dict(model_dict["model"])

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt3,args3 = load_model('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8', MolOpt, args.device)

In [ ]:
list(molopt3.opt0.parameters())[0]

In [ ]:
fgw_loss(*pred_pack, tau = 1)